# CS156 Assignment 2
## Ang Li-Lian

# Data Cleaning

## Variables
For the rejected dataset, I included the amount requested, debt-to-income ratio and employment length. The others were removed because they were irrelevant to whether they would receive the loan (applicant location, date) or were inconsistent and incomparable with the accepted dateset (risk score with both FICO and Vangard Score).

For the accepted dataset, the loan amount, employment length and debt-to-income ratio were chosen because they provide a point of comparison with the rejected dataset. As for the other columns of data,all entries regarding settlement, joint applications, deferrals and hardship plans were removed since they only represent a small subset of the data. They are exceptions to the rule which we won't take into account when making a generalisable model.

In terms of ethical issues, I did not include any data regarding location since the dataset comes from the US where there is a history of housing segregation in certain states. These would have negatively impacted previous data of whether a person was or wasn't accepted for a loan based on their race (which is not a category here, but could be correlated with location). Besides, the person's address intuitively also doesn't affect their ability to pay for a loan and would only obscure other more relevant relationships.

The other columns were chosen based on what would indicate that a person can be trusted to pay back a loan. These include having a good track record of paying on time, having enough money to afford the loan and having a long enough track record. To this end, there are several columns of redundant that are highly correlated and give essentially the same information. For example: "delinq_2yrs" gives the number of 30+ days past-due incidences and "pct_tl_nvr_dlq" gives the percent of trades never delinquent. These two are inversely correlated thereby diluting the effect of variable during analysis especially if many similar ones included. Loan grades are also not included because it already takes into account all of these variables to output a score. Here, we are trying to estimate the risk similar to what these scores would tell us.

### Good track record
- "num_accts_ever_120_pd": number of accounts ever past 120 or more days past due
- "total_rec_late_fee": Late fees received to date
- "pct_tl_nvr_dlq": percentage of trades never delinquent
- "last_fico_range_high": upper boundary of FICO

### Long track record
- "earliest_cr_line" - Earliest reported credit line opened
- "mo_sin_old_il_acct" - Months since oldest bank installment account opened
- "mo_sin_old_rev_tl_op" - Months since oldest revolving account opened 

### Sufficient and stable funds
- "dti": ratio of monthly debt payments to monthly income
- "bc_util": ratio of current balance to credit limit for all bankcard accounts
- "il_util": ratio of total current balance to credit limit on all install accounts
- "inq_fi": number of personal finance inquiries
- "emp_length": employment length
- "home_ownership": home ownership status
- "verification_status": Income verified by LC
- "purpose": reason for loan request

## Dealing with NA's
If there was less than 5% of NAs, all rows with NAs were dropped. The rationale is that this constitutes a very small subset of the data (especially since our dataset is huge), so dropping the data will not affect our sample size. Imputing these entries instead would reduce the variance of our results, giving us more confidence than we should have in our results. To prevent this, we remove the small amount of data.

However, if there was 5% or more NAs, the missing values would be imputed with the mean because removing the data would mean having a significantly smaller data set. Here, we assume that our data is missing at random (rather than having a specific pattern). Therefore, replacing the missing values with the mean would be a good substitute.

## Categorical Data
All categorical data was converted into numerical data using One Hot Encoding that removed the implicit hierarchy simply changing them into ordinal categories would. For Employment Length, categories for "> 10 Years" and "<1 Year" was converted into 10 and 0 respectively and the whole column was converted into integers. This is because there is a hierarchy involved.

## Datetime Data
All datetime data was converted into datetime objects and then into floats so they match with the data type of all other columns.


In [1]:
import pandas as pd 
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = "{:,.2f}".format

import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

# Data source: https://www.kaggle.com/datasets/wordsforthewise/lending-club
accepted = pd.read_csv("accepted_2007_to_2018Q4.csv.gz", nrows= 10000)
accepted = accepted[[ 
 'dti',
 'emp_length',
 'funded_amnt',
 'home_ownership',
 'num_accts_ever_120_pd',
 'pct_tl_nvr_dlq',
 "purpose",
 'bc_util',
 'earliest_cr_line',
 'il_util',
 'inq_fi',
 'last_fico_range_high',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'total_rec_late_fee',
 'verification_status']]

# Transforming Ordinal Data into numeric
accepted[accepted["emp_length"]=="< 1 year"]  ="0"
accepted['emp_length'] = accepted['emp_length'].str.extract('(\d+)', expand=False)
accepted['emp_length'] = accepted['emp_length'].astype(float)

# Removing Missing Data
percent_missing = accepted.isnull().sum() * 100 / len(accepted)

# Very small NAs
drop = percent_missing.index[(percent_missing.values>0)&(percent_missing.values<=5)].tolist()
accepted = accepted.dropna(subset = drop)

# Too significant NAs to drop
fill = percent_missing.index[(percent_missing.values>5)].tolist()
for x in fill:
    accepted[x]=accepted[x].astype(float)
    accepted[x]= accepted[x].fillna(accepted[x].mean())

# Transform categorical value into numeric
for col in ["home_ownership", "verification_status", "purpose"]:
    encode = OneHotEncoder().fit_transform(np.array(accepted[col]).reshape(-1,1)).toarray()
    encode=encode.astype(int)
    accepted[col] = [''.join(map(str, l)) for l in encode]
    accepted[col]=accepted[col].astype(float)

# Transform datetime objects to numeric
for col in  ["earliest_cr_line"]:
    accepted[col]=pd.to_datetime(accepted[col], infer_datetime_format=True, errors= "coerce")
    accepted[col]=accepted[col].values.astype(int)

# Convert all columns to float type
accepted= accepted.astype(float)

In [2]:
# Data Source https://www.kaggle.com/datasets/wordsforthewise/lending-club
rejected = pd.read_csv("rejected_2007_to_2018Q4.csv.gz", nrows= 10000)
rejected = rejected[["Amount Requested", "Employment Length", "Debt-To-Income Ratio"]]

# Rename columns to same name as accepted
rejected = rejected.rename(columns ={"Amount Requested":"funded_amnt", 
    "Employment Length":"emp_length", "Debt-To-Income Ratio":"dti"})

# Transform categorical value into numeric
rejected[rejected["emp_length"]=="< 1 year"]  ="0"
rejected['emp_length'] = rejected['emp_length'].str.extract('(\d+)', expand=False)
rejected['emp_length'] = rejected['emp_length'].astype(int)

# Removing Missing Data
percent_missing = rejected.isnull().sum() * 100 / len(rejected)

# Very small NAs
drop=percent_missing.index[(percent_missing.values>0)&(percent_missing.values<=10)].tolist()
rejected = rejected.dropna(subset = drop)

# Too significant NAs to drop
fill = percent_missing.index[(percent_missing.values>10)].tolist()
rejected[fill]= rejected[fill].fillna(rejected[fill].mean())

# Convert str to float
rejected['dti'] = rejected['dti'].str.rstrip('%').astype('float')

# Convert all columns to float type
rejected=rejected.astype(float)

In [3]:
# Merge and label dataframes
df = pd.concat([rejected,accepted])
df["label"]= [0]*len(rejected)+ [1]*len(accepted)

# Modeling and Evaluation
## Split into train and test sets
The data was randomly split into training and test sets. Note that both data sets also have about the same sample size with the accepted data set being slightly lower because some rows of data were removed, but this is a negligible percentage difference. To prevent leakage of data, the test set was not used at all in any scaling or model preparations. It was only used to find the score of the model. The variable to be predicted was separated out. In the first case it was the funding amount for predicting how much loan a person would receive. The second case is to predict if a person would be accepted or rejected from a loan.

## Scaling Data
A scaler was trained on the training data set and used to transform the training and test data set. To prevent data leakage, the test set was not included to scale the data. 

The data was scaled to normalise and center values which were originally on very different magnitudes. This will ensure that all features contribute proportionally to the final model.


## Accepting or Rejecting a Loan

I've built the following K-Nearest Neigbours model to evaluate whether someone will be accepted or rejected from a loan. Since there are only three overlapping columns between the accepted and rejected database, the number of features is quite small. I tweaked the parameter for class weight so that it adjusts the weights of the data according to the frequency of classes and this gave a marginally better score.

Banks are usually very risk averse because they have a lot at stake to lose if someone is unable to pay back their loan. Therefore, we want our model to minimise the number of false positives (people who should be rejected from a loan, but were accepted) and maximise true positives (accepting people who should be accepted). This means we want a high precision score.

To prevent any leakage in data, we will perform a cross-validation using just the training set and determine which value of K should be used for the K-nearest neighbours to give the highest precision score.

In [4]:
from sklearn.metrics import classification_report,average_precision_score

# Split into training and test data with loan amount as objective
X_train, X_test, y_train, y_test = train_test_split(df[["emp_length","dti"]], df["label"])

# Scale data
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

# search for an optimal value of K for KNN
k_range = range(1, 10)
k_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn, X_train, y_train, cv=10, scoring=['precision_macro'])
    # 4. append mean of scores for k neighbors to k_scores list
    k_scores.append(scores["test_precision_macro"].mean())

K = k_scores.index(max(k_scores))+1 # K with highest precision

neigh = KNeighborsClassifier(n_neighbors=K)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.67      0.69      2480
           1       0.68      0.72      0.70      2442

    accuracy                           0.70      4922
   macro avg       0.70      0.70      0.70      4922
weighted avg       0.70      0.70      0.70      4922



The final precision score is higher for class 0 (rejected) than class 1 (accepted) which shows that the model is doing what we intended which is to maximise the detection of those who should truly be rejected from getting a loan. The scores in general across the board are also fairly high which shows that the model is performing well in predicting who should be given a loan.

## Largest Loan Amount to be Successfully Funded 

Now that we know which applicants have been accepted, we can predict how much loan they will receive. Let's suppose that after screening the applicants, we have a list of all applicants who have been accepted for the loan. In this case, some of our predicted loan applicants might be from the rejected dataset so they won't have any data for other features. For the purposes of this model, let's use the original accepted dataset as a proxy.

We will split the training and test dataset as before, but with the loan amount as our objective. We are also scaling the X_train data to centre and standardise the variance so that all components are given equal consideration. To prevent any data leakage, we will not use X_test until we perform the final test. 

Compared to choosing a regular linear regression, a ridge regression will reduce overfitting in the data by introducing a small amount of bias to improve the performance of the model on unseen data. The alpha value changes how flexible the fit of the curve is. To determine what would be the appropriate alpha value to use we will conduct cross validation on the alpha value for the Ridge regression and select the number of components based on the highest R-squared. Note that we are performing 10-fold cross validation only on the training set, so the test set data will be unseen until the very end, preventing data leakage.

In [5]:
from sklearn.metrics import r2_score,mean_squared_error

# Split data into training and test set
X_train, X_test, y_train, y_test= train_test_split(
        accepted.drop(["funded_amnt"],axis =1),accepted["funded_amnt"])

cols = X_train.columns # feature names

# Scale data
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

# search for an optimal value of K for KNN
k_scores = []

for k in np.arange(0.1,1,0.1):
    reg =  Ridge(alpha=k)
    scores = cross_validate(reg, X_train, y_train, cv=10, scoring=["r2"])
    k_scores.append(scores["test_r2"].mean())

K = k_scores.index(max(k_scores))+1 # K with highest R-squared

In [6]:
reg = Ridge(alpha=K).fit(X_train, y_train) # fit model according to optimised alpha

y_pred = reg.predict(X_test)
test_set_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_set_r2 = r2_score(y_test, y_pred)

print(f"Maximum predicted loan amount: ${'{:.2f}'.format(max(y_pred))}")
print('Root Mean Squared Error:', '{:.2f}'.format(test_set_rmse))
print('Ratio of Mean to RMSE:', '{:.2f}'.format(test_set_rmse/np.mean(y_test)))
print(f"R-Squared: {test_set_r2}")
coeff_df = pd.DataFrame(reg.coef_, cols, columns=['Coefficient'])
coeff_df

Maximum predicted loan amount: $23462.37
Root Mean Squared Error: 7929.00
Ratio of Mean to RMSE: 0.57
R-Squared: 0.2847238001536029


,Coefficient
dti,-3.78
emp_length,33.87
home_ownership,21.10
num_accts_ever_120_pd,-52.09
pct_tl_nvr_dlq,128.76
purpose,-0.00
bc_util,21.23
earliest_cr_line,0.00
il_util,-22.54
inq_fi,130.28


As shown here, we have a predicted maximum loan amount with an R-squared value of about 0.3 which is a fairly low number. This means that the model represents almost 30% of the data which can be considered quite good given the size of our data. To understand how relatively low or high the RMSE is, we will compute its ratio compared to the mean of the actual data. The error is more than 50% of the mean which shows quite a bit of variance. However, since the model was built with careful consideration to not have any leakage from the test set and used a high number of samples to begin with, this model will be generalisable to unseen data.

The coefficients of the Ridge regression also tells us how well each variable describes the model. In this case, 'purpose' and "earliest_cr_line" play next to no role, so they could be removed without any loss to the model itself. But I have left them in the model because when selecting variables, they were intuitvely important to determining loan amount and and approval.